# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.127391e+02     1.404413e+00
 * time: 0.459460973739624
     1     1.192609e+01     8.260435e-01
 * time: 1.08274507522583
     2    -1.150068e+01     9.617794e-01
 * time: 1.1486129760742188
     3    -3.375578e+01     7.360085e-01
 * time: 1.2558250427246094
     4    -4.722922e+01     5.503747e-01
 * time: 1.3334250450134277
     5    -5.670480e+01     2.142005e-01
 * time: 1.438417911529541
     6    -5.966154e+01     1.534305e-01
 * time: 1.494832992553711
     7    -6.083352e+01     4.647775e-02
 * time: 1.5503299236297607
     8    -6.126831e+01     6.148367e-02
 * time: 1.6060960292816162
     9    -6.156283e+01     4.607350e-02
 * time: 1.677954912185669
    10    -6.180548e+01     2.775644e-02
 * time: 1.7339990139007568
    11    -6.199328e+01     2.025088e-02
 * time: 1.7899360656738281
    12    -6.207403e+01     1.698863e-02
 * time: 1.8528690338134766
    13    -6.214141e+01     1.406111e-02
 * time: 1.9088709354400635

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671056
    AtomicLocal         -18.8557633
    AtomicNonlocal      14.8522623
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485355 
    Xc                  -19.3336814

    total               -62.261666458085